In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import torch.optim as optim
from torchinfo import summary
from tqdm import tqdm
from torchvision import models, transforms

In [4]:
x_train = np.load('x_train2.npy')
y_train = np.load('y_train2.npy')
x_train = np.around(x_train,5)
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)
y_train = torch.reshape(y_train,(10000,7))
print('x_train',x_train.shape, x_train.min(), x_train.max())
print('y_train',y_train.shape, y_train.min(), y_train.max())

x_train torch.Size([10000, 1, 40, 40]) tensor(0.) tensor(1.)
y_train torch.Size([10000, 7]) tensor(-0.2000) tensor(0.2000)


In [5]:
# 1. Définir le modèle avec nn.Sequential
class SimpleCNN(nn.Module):
    def __init__(self, output_size):
        super(SimpleCNN, self).__init__()
        self.model = nn.Sequential(
            # Convolutional layer
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            # Flatten layer
            nn.Flatten(),
            # Fully connected layer
            nn.Linear(16 * 20* 20, output_size)
        )
    
    def forward(self, x):
        x = self.model(x)
        x = torch.round(x * 100) / 100# Arrondir les sorties à deux décimales
        return x

# Paramètres
output_size = 7

In [9]:
class ResNet50Custom(nn.Module):
    def __init__(self):
        super(ResNet50Custom, self).__init__()
        self.resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        #Adapter la première couche pour accepter 1 canal au lieu de 3
        self.resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Remplacer la dernière couche fully connected
        self.resnet50.fc = nn.Linear(self.resnet50.fc.in_features, 7) # nombre de para de sortie
    
    def forward(self, x):
        return self.resnet50(x)
    

In [10]:
# Transformation pour adapter la taille d'entrée attendue par ResNet50
transform = transforms.Compose([
    transforms.Resize((224, 224)),
])

# Appliquer la transformation sur les entrées
x_train = torch.stack([transform(input) for input in x_train])

In [11]:
model = ResNet50Custom()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)#Adam(model.parameters(), lr=0.005)#

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /home/zita1/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 76.6MB/s]


In [12]:
model(x_train)

: 

In [ ]:
# 4. Boucle d'entraînement avec enregistrement des pertes
epochs = 500
losses = []
with tqdm(range(epochs), unit='epoch') as tepoch :
    for epoch in tepoch:
        

        # Forward pass
        outputs = model(x_train)
        loss = criterion(outputs, y_train)
        losses.append(loss.item())
        optimizer.zero_grad () # clean up step
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        tepoch.set_postfix(loss=loss.item())

100%|██████████| 5000/5000 [1:07:40<00:00,  1.23epoch/s, loss=0.0158]


In [1]:
# 5. Tracer le graphique de la perte d'entraînement
plt.plot(losses)
plt.xlabel('Époque')
plt.ylabel('Perte')
plt.title('Perte d\'entraînement au fil des époques')
plt.show()

NameError: name 'plt' is not defined

In [2]:
with torch.no_grad():
    # Extraire la deuxième image de inputs
    test_input = x_train[0].unsqueeze(0)  # Ajouter une dimension batch
    predicted_output = model(test_input)
print(predicted_output)


NameError: name 'torch' is not defined